# Scrape Missing Values

Some of the columns are missing values that are available in [pokemondb.net](https://pokemondb.net/pokedex).

This notebook uses our custom scraper to retrieve missing values from the website.

## Setup

In [1]:
import pandas as pd

from scrape import Variant

In [2]:
IN_PATH = "pokedex_(Update_04.21).csv"
OUT_PATH = "filled.csv"

pokedex = pd.read_csv(IN_PATH)

## Scraping

In [3]:
NAME = "name"
CATCH_RATE = "catch_rate"
BASE_FRIENDSHIP = "base_friendship"
BASE_EXPERIENCE = "base_experience"

missing_catch_rate = pd.isna(pokedex[CATCH_RATE])
missing_base_friendship = pd.isna(pokedex[BASE_FRIENDSHIP])
missing_base_experience = pd.isna(pokedex[BASE_EXPERIENCE])
missing_any = (
    missing_catch_rate |
    missing_base_friendship |
    missing_base_experience
)

variants_to_fetch = pokedex.loc[
    missing_any,
    ["name", CATCH_RATE, BASE_FRIENDSHIP, BASE_EXPERIENCE]
]
variants_to_fetch

,name,catch_rate,base_friendship,base_experience
69,Galarian Meowth,NaN,NaN,NaN
100,Galarian Ponyta,NaN,NaN,NaN
102,Galarian Rapidash,NaN,NaN,NaN
107,Galarian Slowbro,75.0,70.0,NaN
111,Galarian Farfetch'd,NaN,NaN,NaN
...,...,...,...,...
1040,Glastrier,3.0,NaN,NaN
1041,Spectrier,3.0,NaN,NaN
1042,Calyrex,3.0,NaN,NaN
1043,Calyrex Ice Rider,3.0,NaN,NaN


In [4]:
fetched_variants = {
    i: Variant.fetch(variant_name)
    for i, variant_name in zip(variants_to_fetch.index, variants_to_fetch[NAME])
}
fetched_variants

{69: <scrape.scrape.Variant at 0x7f55c812b700>,
 100: <scrape.scrape.Variant at 0x7f55c812b880>,
 102: <scrape.scrape.Variant at 0x7f55c812ba60>,
 107: <scrape.scrape.Variant at 0x7f5593b8b7c0>,
 111: <scrape.scrape.Variant at 0x7f5593bd9f00>,
 144: <scrape.scrape.Variant at 0x7f5593c30700>,
 158: <scrape.scrape.Variant at 0x7f5593c5aef0>,
 185: <scrape.scrape.Variant at 0x7f5591cd9300>,
 187: <scrape.scrape.Variant at 0x7f5591d31f60>,
 189: <scrape.scrape.Variant at 0x7f5591d949a0>,
 246: <scrape.scrape.Variant at 0x7f5591bbec50>,
 273: <scrape.scrape.Variant at 0x7f5591c0cf70>,
 320: <scrape.scrape.Variant at 0x7f5591c3b220>,
 322: <scrape.scrape.Variant at 0x7f5591c8d7e0>,
 654: <scrape.scrape.Variant at 0x7f5591abbd30>,
 657: <scrape.scrape.Variant at 0x7f5591b16080>,
 658: <scrape.scrape.Variant at 0x7f5591b683d0>,
 666: <scrape.scrape.Variant at 0x7f5591b8e440>,
 723: <scrape.scrape.Variant at 0x7f55919e44f0>,
 944: <scrape.scrape.Variant at 0x7f5591a16680>,
 945: <scrape.scrape.

In [5]:
filled = pokedex.copy()

for i, variant in fetched_variants.items():
    variant_name = variant.variant_name
    assert filled.at[i, NAME] == variant_name
    print(variant_name)
    
    catch_rate = variant.catch_rate
    if pd.isna(filled.at[i, CATCH_RATE]):
        print(f"\t{catch_rate=}")
        filled.at[i, CATCH_RATE] = catch_rate
    elif filled.at[i, CATCH_RATE] != catch_rate:
        print(f"\tExpected {catch_rate=}, got {filled.at[i, CATCH_RATE]}")
        
    base_friendship = variant.base_friendship
    if pd.isna(filled.at[i, BASE_FRIENDSHIP]):
        print(f"\t{base_friendship=}")
        filled.at[i, BASE_FRIENDSHIP] = base_friendship
    elif filled.at[i, BASE_FRIENDSHIP] != base_friendship:
        print(f"\tExpected {base_friendship=}, got {filled.at[i, BASE_FRIENDSHIP]}")
        
    base_experience = variant.base_experience
    if pd.isna(filled.at[i, BASE_EXPERIENCE]):
        print(f"\t{base_experience=}")
        filled.at[i, BASE_EXPERIENCE] = base_experience
    elif filled.at[i, BASE_EXPERIENCE] != base_experience:
        print(f"\tExpected {base_experience=}, got {filled.at[i, BASE_EXPERIENCE]}")

Galarian Meowth
	catch_rate=255
	base_friendship=50
	base_experience=58
Galarian Ponyta
	catch_rate=190
	base_friendship=50
	base_experience=82
Galarian Rapidash
	catch_rate=60
	base_friendship=50
	base_experience=175
Galarian Slowbro
	Expected base_friendship=50, got 70.0
	base_experience=172
Galarian Farfetch'd
	catch_rate=45
	base_friendship=50
	base_experience=132
Galarian Weezing
	catch_rate=60
	base_friendship=50
	base_experience=172
Galarian Mr. Mime
	catch_rate=45
	base_friendship=50
	base_experience=161
Galarian Articuno
	base_experience=290
Galarian Zapdos
	base_experience=290
Galarian Moltres
	base_experience=290
Galarian Slowking
	Expected base_friendship=50, got 70.0
	base_experience=172
Galarian Corsola
	catch_rate=60
	base_friendship=50
	base_experience=144
Galarian Zigzagoon
	catch_rate=255
	base_friendship=50
	base_experience=56
Galarian Linoone
	catch_rate=90
	base_friendship=50
	base_experience=147
Galarian Darumaka
	catch_rate=120
	base_friendship=50
	base_experienc

## Save Results

In [6]:
filled.to_csv(OUT_PATH)